In [1]:
!pip install tensorflow==2.13.0 opencv-python mediapipe pygame numpy

In [2]:
import sys
import cv2
import numpy as np
import os
import time
import mediapipe as mp
import pygame

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [7]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect. actions[0] == 'Guru1', actions[1] == 'Guru2' and so on...
actions = np.array(['Guru1', 'Guru2', 'Guru3', 'Guru4', 'Guru5', 'Guru6', 'Guru7', 'Guru8', 'Stand'])

# Videos are going to be 30 frames in length
sequence_length = 30

In [8]:
label_map = {label:num for num, label in enumerate(actions)}

In [9]:
label_map

{'Guru1': 0,
 'Guru2': 1,
 'Guru3': 2,
 'Guru4': 3,
 'Guru5': 4,
 'Guru6': 5,
 'Guru7': 6,
 'Guru8': 7,
 'Stand': 8}

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [11]:
# Once training is done, u do not have to train it again (no need to call the fit() finction). Just need to setup the model
# and compile it. then load the saved weights.
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [12]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [13]:
model.load_weights('action.h5')

# Prediction Cell

In [14]:
import time
sequence = []
threshold = 0.9
threshold1 = 0.8
threshold2 = 0.6

pygame.init()
pygame.mixer.init()
pose_achieved=pygame.mixer.Sound("pach.wav")
introduction=pygame.mixer.Sound("introduction.wav")
instructions = [None] * 8
instructions[0]=pygame.mixer.Sound("instruct1.wav")
instructions[1]=pygame.mixer.Sound("instruct2.wav")
instructions[2]=pygame.mixer.Sound("instruct3.wav")
instructions[3]=pygame.mixer.Sound("instruct4.wav")
instructions[4]=pygame.mixer.Sound("instruct5.wav")
instructions[5]=pygame.mixer.Sound("instruct6.wav")
instructions[6]=pygame.mixer.Sound("instruct7.wav")
instructions[7]=pygame.mixer.Sound("instruct8.wav")

BeepOnePerSec=pygame.mixer.Sound("BeepOnePerSec.wav")
BeepTwoPerSec=pygame.mixer.Sound("BeepTwoPerSec.wav")

startatstep = 0
stopatstep = 2
step = startatstep

# Once we call instructions.play() - the audio will start playing. And the code will move on to the
# next line.
introduction.play()
time.sleep(10)
instructions[step].play()
time.sleep(10)


cap = cv2.VideoCapture(0)
# Set mediapipe model, note  that we set a high bar for detection confidence. this is just
# saying that, we want the model to be absolutely sure that it detected a pose (with landmarks)
# before it claims victory. we would rather not  have a "match" than have it approximate. the
# goal here is to be as close to the pose as possible!
with mp_holistic.Holistic(min_detection_confidence=0.95) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections, this will call the model with the actions.h5 data from the training
        # and send us an array saying what was the match ratio for each of the poses it knows about.
        image, results = mediapipe_detection(frame, holistic)
        # print(results)
        
        # Draw landmarks, so the user can see it on screen. This won't be helpful for a blind user
        # this is just here for the developers/us to make sure the code is working correctly and 
        # landmarks are being detected.
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)
        
        # If we have captured 30 frames of data, use the last 30 frames for prediction
        # this is what makes our model detect the fluidity of motion... it will detect the way
        # we train it... 30 frames == 1 sec. So, our pose can be a 1 second video - which will be
        # matched with the trained data.
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0), verbose=0)[0]
            if res[step] > 0.4 and res[step] < 0.6:
                print(res[step])
            # Did we have a match?
            if res[step] > threshold:
                print(res)
                pose_achieved.play()
                cv2.waitKey(5000)
                # wait 10 seconds and then go to next step.
                step = step + 1
                if step <= stopatstep:
                    instructions[step].play()
                    time.sleep(10)
                    continue
                else:
                    # All steps are done. break from loop
                    break
            # Did we have a close match? If yes, beep faster.
            elif res[step] > threshold1:
                print(res)
                BeepTwoPerSec.play()
                cv2.waitKey(1000)
                BeepTwoPerSec.stop()
            # Did we have a rough match? Beep slower, so the user knows they are getting close.
            elif res[step] > threshold2:
                print(res)
                BeepOnePerSec.play()
                cv2.waitKey(1000)
                BeepOnePerSec.stop()
                
        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

[2.1404180e-01 1.3338355e-07 8.2631692e-07 3.7720118e-05 2.9862813e-05
 2.9227638e-04 7.6838342e-06 2.3293627e-09 7.8558975e-01]
[4.9571562e-01 6.0584853e-08 1.8388080e-07 1.8970408e-05 7.6508361e-05
 1.8465272e-04 4.0487471e-06 3.7365684e-09 5.0400001e-01]
[9.5904309e-01 6.4946531e-10 4.0780723e-10 3.0714978e-07 6.1543571e-05
 7.0729252e-06 8.2108693e-07 1.0751998e-09 4.0887121e-02]
[9.5904309e-01 6.4946531e-10 4.0780723e-10 3.0714978e-07 6.1543571e-05
 7.0729252e-06 8.2108693e-07 1.0751998e-09 4.0887121e-02]
[4.23406251e-03 1.46674089e-15 1.36656843e-12 1.04185585e-08
 9.95658398e-01 8.12389942e-07 8.27679347e-09 3.68473501e-08
 1.06732594e-04]
[8.1135369e-07 1.2799542e-17 2.5128962e-14 3.2227683e-09 9.9999917e-01
 3.2743876e-09 4.4810107e-11 3.6723847e-08 7.6978202e-10]
[6.0811772e-08 6.7169884e-23 1.6615664e-19 1.8524654e-12 9.9999988e-01
 2.5701999e-10 6.3800168e-14 2.4688110e-10 7.9142881e-12]
[6.8825091e-07 3.2046739e-24 9.5227633e-21 3.4661887e-13 9.9999928e-01
 3.6261233e-10 1

[7.6032877e-01 4.9462583e-04 1.9843792e-04 1.7749488e-02 2.0836765e-01
 6.8302766e-06 1.0081647e-04 1.5799903e-10 1.2753357e-02]
[7.6474983e-01 5.0638325e-04 2.0263725e-04 1.8571870e-02 2.0319508e-01
 6.6392031e-06 9.8829121e-05 1.4766303e-10 1.2668652e-02]
[7.66285956e-01 5.10822167e-04 2.00589202e-04 1.83827262e-02
 2.01882809e-01 6.63246419e-06 9.85688675e-05 1.46979109e-10
 1.26319425e-02]
[7.6542199e-01 5.0273724e-04 1.9453348e-04 1.8573865e-02 2.0281896e-01
 6.6198199e-06 9.5908770e-05 1.4167517e-10 1.2385422e-02]
[7.6164943e-01 4.8807959e-04 1.8464736e-04 1.8397786e-02 2.0707560e-01
 6.7591845e-06 9.3847484e-05 1.4169098e-10 1.2103975e-02]
[7.5134212e-01 4.5564052e-04 1.6978387e-04 1.8184109e-02 2.1804956e-01
 7.0137144e-06 9.0414193e-05 1.4425468e-10 1.1701354e-02]
[7.22801566e-01 3.80969723e-04 1.47653001e-04 1.80918090e-02
 2.47460410e-01 7.20995968e-06 8.25901443e-05 1.46890750e-10
 1.10277785e-02]
[6.5994650e-01 2.5257736e-04 1.1779828e-04 1.8506274e-02 3.1125897e-01
 7.215

[6.3140199e-26 3.8832573e-24 5.7138603e-21 1.6303663e-04 3.9053171e-13
 7.6328769e-02 4.3317610e-03 9.1917640e-01 2.1876743e-20]
[1.02090496e-25 1.47092431e-22 2.15218663e-20 5.09248115e-04
 1.44571755e-16 9.23727334e-01 7.51248226e-02 6.38586818e-04
 4.71562691e-19]
[2.9618632e-22 9.1613734e-21 1.2526468e-19 3.9173577e-02 3.1111346e-12
 6.4934796e-01 2.9058644e-01 2.0892017e-02 1.2601397e-16]
[5.5132593e-24 2.3196209e-22 3.8864018e-20 8.7838851e-02 1.0448854e-14
 7.6109624e-01 1.5047999e-01 5.8492692e-04 6.1952704e-17]
[1.9074728e-25 3.8725697e-22 3.0190955e-19 2.0388590e-01 1.4629793e-17
 2.6479626e-01 5.3131580e-01 2.1135722e-06 7.1579520e-18]
[3.26448689e-28 6.90544493e-26 1.15982596e-22 2.71143335e-06
 6.52243777e-22 3.16705137e-01 6.83292210e-01 9.73554570e-11
 1.30618935e-20]
[6.3842128e-31 2.3939121e-30 5.9041992e-26 5.5094942e-11 2.1439635e-25
 8.2226717e-01 1.7773281e-01 1.6795525e-13 1.0707546e-22]
[2.9615645e-37 3.0494525e-35 9.0370448e-32 6.9281810e-23 4.4345392e-36
 9.996

[1.1914464e-03 1.6719085e-14 4.9610867e-12 1.5092955e-07 9.9878734e-01
 1.0252149e-07 2.3465052e-09 5.0284399e-08 2.0788530e-05]
[1.6026848e-03 4.3288757e-14 9.7170275e-12 2.3089400e-07 9.9836689e-01
 1.7691495e-07 3.8699675e-09 6.6297879e-08 2.9950632e-05]
[1.7151885e-03 8.3396499e-14 1.4363456e-11 3.0970293e-07 9.9825007e-01
 3.2573800e-07 6.1667178e-09 9.0108237e-08 3.4033841e-05]
[1.34087075e-03 7.89709891e-14 1.14026115e-11 2.96930466e-07
 9.98634398e-01 4.47601792e-07 6.64870958e-09 9.59617665e-08
 2.39037181e-05]
[7.2328281e-04 6.7705349e-14 9.1981501e-12 2.9814211e-07 9.9926513e-01
 5.6964478e-07 7.1990960e-09 1.2866455e-07 1.0622305e-05]
[5.4791383e-04 6.2798060e-14 7.6408515e-12 2.7675648e-07 9.9944443e-01
 6.6619322e-07 7.2783468e-09 1.4784958e-07 6.5700578e-06]
[3.6816191e-04 4.7760224e-14 5.1391005e-12 2.2796829e-07 9.9962759e-01
 7.1497880e-07 6.2639609e-09 1.3928474e-07 3.0642786e-06]
[2.7797319e-04 3.9966941e-14 3.9790441e-12 1.9058712e-07 9.9971920e-01
 8.4806788e-07 5

[7.2475833e-01 1.4135120e-06 6.6186686e-07 1.6500149e-04 2.0318168e-04
 1.6980886e-04 2.3551345e-05 1.7317534e-08 2.7467802e-01]
[7.18929619e-02 1.59180686e-07 5.25843404e-07 7.15336573e-05
 1.01779215e-05 1.33171605e-04 1.15844314e-05 3.47197138e-09
 9.27879930e-01]
[3.6790001e-03 6.5170074e-09 8.4391424e-08 4.9557962e-06 3.5873862e-07
 1.2489217e-05 5.6432447e-07 3.6051034e-11 9.9630249e-01]
[2.1105160e-03 5.7736460e-09 1.0769985e-07 2.8769566e-06 2.3101533e-07
 5.7483821e-06 2.1911531e-07 1.1039190e-11 9.9788034e-01]
[3.3877909e-04 1.9342530e-09 1.1203451e-07 7.1829015e-07 2.3679443e-09
 1.1670513e-06 1.0011434e-08 3.8393669e-14 9.9965918e-01]
[1.0520584e-03 1.8012658e-08 1.2033178e-06 1.5865190e-06 2.7224614e-10
 8.7686527e-07 2.5495590e-09 3.5213173e-15 9.9894422e-01]
[9.8760333e-04 2.4430539e-08 1.7583723e-06 1.6677413e-06 2.7596464e-10
 7.4189512e-07 2.5617237e-09 3.8064759e-15 9.9900824e-01]
[4.1368938e-04 9.0083798e-09 7.6690532e-07 9.9527381e-07 5.3200183e-10
 6.9274546e-07 4

...

[9.9999988e-01 1.2169287e-09 5.1348309e-12 5.9167526e-08 4.0079436e-16
 3.0268845e-12 3.5500556e-16 1.4792356e-22 8.7287863e-14]
[1.0000000e+00 3.7066125e-10 1.4043231e-12 1.8078323e-08 2.1644343e-16
 2.2425228e-12 8.4111444e-17 3.4830535e-23 9.7207791e-14]
[1.0000000e+00 1.2138912e-10 3.5494456e-13 3.9201509e-09 1.1147590e-16
 1.5638233e-12 1.6442225e-17 6.9161818e-24 9.1697537e-14]
[1.0000000e+00 3.9451095e-11 7.6075194e-14 6.3784861e-10 4.5806897e-17
 8.5219722e-13 2.3834289e-18 9.6615749e-25 7.2397736e-14]
[1.00000000e+00 1.13865384e-11 1.50449789e-14 8.23697499e-11
 1.29028934e-17 3.25098482e-13 3.21363313e-19 7.33397512e-26
 5.32725888e-14]
[1.0000000e+00 2.5125848e-12 2.2052399e-15 1.0103974e-11 2.4045886e-18
 9.3643179e-14 3.8573510e-20 3.6292081e-27 2.4339036e-14]
[1.0000000e+00 6.6992635e-13 3.7865766e-16 1.6523189e-12 5.3800163e-19
 3.1481674e-14 6.1219395e-21 2.6481132e-28 1.1170972e-14]
[1.0000000e+00 2.1610839e-13 8.3451466e-17 4.1209408e-13 1.6870694e-19
 1.2732101e-14 1

[9.9999976e-01 2.2294734e-07 1.7329879e-10 4.7593312e-09 2.7747719e-15
 3.2415557e-12 8.0014242e-16 7.8711365e-21 6.8047273e-15]
[9.9999988e-01 7.5748773e-08 4.5201055e-11 2.2615418e-09 9.3583260e-16
 1.7639303e-12 2.5224999e-16 1.1438591e-21 5.1171755e-15]
[1.0000000e+00 2.9621809e-08 1.4522495e-11 1.0945911e-09 3.7878623e-16
 1.0979517e-12 9.2574838e-17 2.3029054e-22 4.2121009e-15]
[1.0000000e+00 1.3328238e-08 5.6146411e-12 5.7244248e-10 1.8280118e-16
 7.4831683e-13 3.8392562e-17 6.1173109e-23 3.6243600e-15]
[1.0000000e+00 6.6668999e-09 2.3751181e-12 2.9884645e-10 9.4082914e-17
 5.2475418e-13 1.6320308e-17 1.8121408e-23 3.0369773e-15]
[1.0000000e+00 3.4732663e-09 1.0785550e-12 1.6442975e-10 5.1433753e-17
 3.7487973e-13 7.2991162e-18 5.9821693e-24 2.5546806e-15]
[1.0000000e+00 2.4429885e-09 7.9441092e-13 1.4359444e-10 4.9804164e-17
 3.8142726e-13 5.7282679e-18 4.5149239e-24 3.2335239e-15]
[1.0000000e+00 1.9603026e-09 6.7856831e-13 1.4106548e-10 6.1446463e-17
 4.5111186e-13 5.5686994e-

[2.1592502e-12 9.9999940e-01 5.6888052e-07 2.4036279e-15 5.7252577e-23
 1.8576636e-22 2.5650824e-20 4.4313652e-23 4.7605464e-24]
[1.9203012e-12 9.9999940e-01 5.7396272e-07 2.4368644e-15 6.6265850e-23
 2.2338569e-22 3.2150397e-20 6.3226488e-23 5.0012807e-24]
[1.8077793e-12 9.9999940e-01 6.2526510e-07 2.8931636e-15 9.4491886e-23
 3.0030034e-22 4.5911064e-20 1.1168705e-22 6.0848143e-24]
[1.6518516e-12 9.9999940e-01 6.1065862e-07 2.6733311e-15 9.2698187e-23
 3.0446903e-22 4.7931918e-20 1.2652180e-22 5.5720370e-24]
[1.4824397e-12 9.9999940e-01 5.6228185e-07 2.0977775e-15 7.0911944e-23
 2.4566242e-22 3.9739534e-20 1.1029830e-22 4.0746303e-24]
[1.6099006e-12 9.9999940e-01 6.5026836e-07 2.9971886e-15 1.2527790e-22
 3.9644862e-22 6.5384336e-20 2.1022161e-22 6.5360763e-24]
[1.8460379e-12 9.9999928e-01 7.5759453e-07 4.4404186e-15 2.6527602e-22
 8.1976461e-22 1.3396663e-19 5.2299782e-22 1.2344902e-23]
[2.0648552e-12 9.9999917e-01 8.8808764e-07 6.3854993e-15 4.9172338e-22
 1.4012010e-21 2.3292085e-

[7.5435421e-13 9.9999988e-01 7.5269043e-08 3.2317937e-17 7.9509377e-26
 6.5535029e-25 7.3686184e-23 1.7786747e-26 1.8164427e-26]
[8.9504923e-13 9.9999988e-01 1.4885842e-07 1.3195789e-16 3.1753473e-25
 1.3787452e-24 2.0155837e-22 6.0508084e-26 6.1865776e-26]
[3.7157004e-14 1.0000000e+00 2.9113087e-09 1.3186950e-21 5.0749277e-32
 9.1495011e-30 9.5871040e-28 1.4795611e-32 5.2624954e-32]
[2.4580050e-13 9.9999988e-01 9.3064408e-08 6.8627983e-18 1.0347917e-27
 4.9122939e-27 1.2509924e-24 2.7592377e-28 3.4611763e-28]
[8.9988933e-15 1.0000000e+00 1.1063707e-09 5.9008934e-24 1.3552823e-35
 6.7441696e-33 4.7439623e-31 5.7148580e-36 1.1005093e-35]
[2.6800785e-14 1.0000000e+00 4.2316515e-08 9.1877011e-21 1.6635787e-32
 1.6715503e-31 3.4162218e-29 1.4593590e-32 5.3984600e-33]
[7.4274159e-10 2.0536330e-01 7.9463309e-01 3.5436269e-06 3.2891409e-16
 2.3720544e-16 7.7733111e-15 2.3965424e-13 2.3096667e-16]
[4.7719776e-14 9.9999523e-01 4.7939743e-06 7.8705377e-19 1.8084302e-31
 1.5659497e-31 5.9496045e-

[1.2009712e-12 9.9999690e-01 3.0728161e-06 1.1584513e-15 2.0427760e-24
 3.2968592e-24 7.1204282e-22 9.7452421e-24 4.3375548e-26]
[1.61931297e-12 9.99998212e-01 1.79576830e-06 7.74403048e-16
 2.90023016e-24 7.24711604e-24 1.30253916e-21 1.04401465e-23
 9.43022199e-26]
[2.7684487e-12 9.9999785e-01 2.1291212e-06 1.7255491e-15 1.4102389e-23
 4.0132611e-23 6.0982223e-21 5.3489509e-23 5.1114030e-25]
[6.1158596e-12 9.9999738e-01 2.6620778e-06 7.0071504e-15 1.5738174e-22
 3.8307954e-22 5.2322880e-20 5.1257990e-22 5.8091938e-24]
[1.4497641e-11 9.9999607e-01 3.9041802e-06 3.6972344e-14 2.5316804e-21
 5.0009094e-21 6.2266124e-19 6.8287871e-21 9.5360277e-23]
[6.3937466e-12 9.9999893e-01 1.1109026e-06 1.4355339e-15 5.5078982e-23
 4.8253084e-22 4.6526890e-20 1.8651469e-22 4.0811616e-24]
[4.2082830e-11 9.9999142e-01 8.5686097e-06 5.2123735e-13 1.8924415e-19
 2.4418276e-19 2.8046601e-17 5.4844696e-19 5.4958151e-21]
[3.8923021e-11 9.9999750e-01 2.4851047e-06 7.3533586e-14 2.2301767e-20
 7.3863132e-20 6